## Load Routines for Printing Matrices

In [ ]:
import Formatting              #do Pkg.add("Formatting") to install this package
include("printmat.jl")

# Bond Return after a Sudden Interest Rate Change

In [18]:
m  = 5
θ₋ = 0.02                   #initial interest rate
θ₊ = 0.03                   #interest rate after sudden change
B₋ = (1+θ₋)^(-m)
B₊ = (1+θ₊)^(-m)

PriceChange = B₊ - B₋

println("Bond price before and after change of interest from $θ₋ to $θ₊: ",round([B₋ B₊],3))

println("Exact price change: ",round(PriceChange,3))

Bond price before and after change of interest from 0.02 to 0.03: [0.906 0.863]
Exact price change: -0.043


# From Ch 14

In [19]:
function BondPrice3Ps(Y,cf,mk)              #cf is a vector of all cash flows
    if length(Y)==1 & length(mk)>1          #to handle case with Y being a scalar
        Y = fill(Y,length(mk))      
    end    
    cfac = cf./((1+Y).^mk)                  #c/(1+y1)^t1 + c/(1+y2)^t2 + ...+ c/(1+y2)^m
    P    = sum(cfac)                        #price
    return P
end

P = BondPrice3Ps(θ₋,[zeros(4);1],1:5)
println("Price of bond with 0% coupon: ", round(P,3))

using Roots                #package for solving non-linear equations

ytm = fzero(y->BondPrice3Ps(y,[zeros(4);1],1:5)-P,[-0.1;0.1])
println("ytm: ",round(ytm,3))

Price of bond with 0% coupon: 0.906
ytm: 0.02


# Macaulay's Duration

$D^M = \sum_{k=1}^{K} m_{k}\frac{cf_{k}}{\left(  1+\theta\right)  ^{m_{k}}P}$

A first-order Taylor approximation gives

$\frac{\Delta P}{P} = -D^M \times \frac{\Delta \theta}{1+\theta}$ 

In [20]:
function BondDurationMacaulay3Ps(P,cf,mk,ytm)
    cfac    = cf.*mk./((1+ytm).^(mk+1))     #c/(1+y)^2 + 2c/(1+y)^3 + ...+ Kc/(1+y)^(K+1)
    D       = sum(cfac)                     #Duration, row vector
    Da      = D/P                           #adjusted duration
    Dmac    = D*(1+ytm)/P                   #Macaulays duration
    return D,Da,Dmac
end

(D,Da,Dmac) = BondDurationMacaulay3Ps(P,[zeros(4);1],1:5,ytm)

println("Dollar duration and Macaulay's duation: ",round([D Dmac],3))

Δθ = θ₊ - θ₋

ΔP    = -D*Δθ
ΔPrel = -Dmac*Δθ/(1+θ₋)
println("approximate price change according to dollar duration, relative price change accorrding to Macaulay's: ")
println(round([ΔP ΔPrel],3))

Dollar duration and Macaulay's duation: [4.44 5.0]
approximate price change according to dollar duration, relative price change accorrding to Macaulay's: 
[-0.044 -0.049]


In [21]:
(D,Da,Dmac) = BondDurationMacaulay3Ps(1.019,[0.04;0.04+1],1:2,0.03)
println("Durations for 2-year 4% bond with ytm = 0.03: ",round([D Da Dmac],3))

Durations for 2-year 4% bond with ytm = 0.03: [1.941 1.905 1.962]


# Hedging a Liability Stream

We are short one $L$ bond, which is worth $P_L$. To hedge that we buy $h$ units of bond $H$. The balance is put on a money market account $M$ to make the initial value of the portfolio zero ($V=0$)

$V=hP_{H}+M-P_{L}$

Over a short time interval, the change in the portfolio value is

$\Delta V=h\Delta P_{H}-\Delta P_{L}$, which can be approximated as

$\Delta V  =-hD_{H}^{M}P_{H}\times\frac{\Delta\theta_{H}}{1+\theta_{H}}+D_{L}^{M} P_{L}\times\frac{\Delta\theta_{L}}{1+\theta_{L}}$

In [22]:
cf = [150;250]                   #cash flow of liability stream (as positive numbers)
mk = [1;2]                       #times of cash flows   
θ₋ = 0.05                        #initial interest rate
θ₊ = 0.04                        #interest rate after sudden change
Δθ = θ₊ - θ₋

PL₋ = BondPrice3Ps(θ₋,cf,mk)      #price at initial interest rate, no face value  
PL₊ = BondPrice3Ps(θ₊,cf,mk)      #price at new interest rate

println("Value of liability at interest rates $θ₋ and $θ₊: ",round([PL₋ PL₊],3))

(_,_,Dmac) = BondDurationMacaulay3Ps(PL₋,cf,mk,θ₋)
println("Macaulay's duration: ",round(Dmac,3))

Value of liability at interest rates 0.05 and 0.04: [369.615 375.37]
Macaulay's duration: 1.613


### Naive Hedging with Duration of Hedge Bond DH = 0.5 (flat yield curve) 

set $h=P_{L}/P_{H}$ to get

$\frac{\Delta V}{P_{L}}\approx\left(  D_{L}^{M}-D_{H}^{M}\right)  \times
\frac{\Delta\theta}{1+\theta}$

In [23]:
PH₋ = BondPrice3Ps(θ₋,collect(100),collect(0.5))             #collect() to mae them vectors
PH₊ = BondPrice3Ps(θ₊,collect(100),collect(0.5))      
h = PL₋/PH₋
println("A zero coupon bond with m=0.5 and face value of 100: price and hedge ratio h: ", round([PH₋ h],3))

ΔV_exact = h*(PH₊-PH₋) - (PL₊-PL₋)
println("(relative) change of hedge portfolio value (exact): ",round(ΔV_exact/PL₋,4))
ΔVrel_approx = (Dmac - 0.5)*Δθ/(1+θ₋)
println("(relative) change of hedge portfolio value (approx): ",round(ΔVrel_approx,4))

A zero coupon bond with m=0.5 and face value of 100: price and hedge ratio h: [97.59 3.787]
(relative) change of hedge portfolio value (exact): -0.0108
(relative) change of hedge portfolio value (approx): -0.0106


### Duration Hedging  (flat yield curve)

instead, set 

$h =\frac{D_{L}^{M}}{D_{H}^{M}}\times\frac{P_{L}}{P_{H}}$ to get

$\frac{\Delta V}{P_{L}}\approx D_{L}^{M}\times\left(  \frac{\Delta\theta_{L} }{1+\theta_{L}}-\frac{\Delta\theta_{H}}{1+\theta_{H}}\right)$

In [24]:
PH₋ = BondPrice3Ps(θ₋,collect(100),collect(Dmac))             
PH₊ = BondPrice3Ps(θ₊,collect(100),collect(Dmac))      
h = Dmac/Dmac * PL₋/PH₋
println("A zero coupon bond with m=$Dmac and face value of 100: price and hedge ratio h: ", round([PH₋ h],3))

ΔV_exact = h*(PH₊-PH₋) - (PL₊-PL₋)
println("(relative) change of hedge portolio value (exact): ",round(ΔV_exact/PL₋,6))
ΔVrel_approx = Dmac*(Δθ/(1+θ₋) - Δθ/(1+θ₋)) 
println("(relative) change of hedge portolio value (approx): ",round(ΔVrel_approx,6))

A zero coupon bond with m=1.6134969325153377 and face value of 100: price and hedge ratio h: [92.43 3.999]
(relative) change of hedge portolio value (exact): -1.1e-5
(relative) change of hedge portolio value (approx): 0.0


# Hedging a Liability II (Example from Figure in the Lecture Notes)

In [25]:
cf = ones(10)*0.2               #cash flow of liability stream (as positive numbers)
mk = 1:10                       #times of cash flows   
θ₋ = 0.05                       #initial interest rate
θ₊ = 0.03                       #interest rate after sudden change
Δθ = θ₊ - θ₋

PL₋ = BondPrice3Ps(θ₋,cf,mk)       #price at initial interest rate
PL₊ = BondPrice3Ps(θ₊,cf,mk)       #price at new interest rate
println("Value of liability at interest rates $θ₋ and $θ₊: ",round([PL₋ PL₊],3))

(_,_,Dmac) = BondDurationMacaulay3Ps(PL₋,cf,mk,θ₋)
println("Duration: ",round(Dmac,3))

PH₋ = BondPrice3Ps(θ₋,collect(1),collect(3))             #collect() to makle them "vectors"
PH₊ = BondPrice3Ps(θ₊,collect(1),collect(3))      
h = Dmac/3 * PL₋/PH₋
println("\nValue of hedge bond with m=3 at interest rates $θ₋ and $θ₊: ",round([PH₋ PH₊],3))
println("hedge ratio h: ", round(h,3))

Value of liability at interest rates 0.05 and 0.03: [1.544 1.706]
Duration: 5.099

Value of hedge bond with m=3 at interest rates 0.05 and 0.03: [0.864 0.915]
hedge ratio h: 3.039
